In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt

import os

import h5py
import numpy as np
import seaborn as sns

from astropy import constants as const
from astropy import units as u

from injection_helpers import snapshot_to_energy_file

 
# Note: these units differ from the ones in `units.py`
# These match the code units (M_sun, pc, Myr), while `units.py` uses cgs

m_p = const.m_p.cgs.to(u.M_sun).value

cm = u.cm.to(u.pc)
g = u.g.to(u.M_sun)

initial_metallicity = 0.02 # gas-phase mas fraction

In [ ]:
run_name = "cluster_cooling_250"
inputs_dir  = os.path.join("../runs", run_name, "inputs")
outputs_dir = os.path.join("../runs", run_name, "outputs")

IC_filename = os.path.join(inputs_dir, "{}_ics.hdf5".format(run_name))

print(inputs_dir)
print(outputs_dir)

print(IC_filename)

# Create cluster initial conditions

In [ ]:
ic_file = h5py.File(IC_filename, "w")

In [ ]:
header_group = ic_file.create_group("Header")
particles_group = ic_file.create_group("PartType0")

In [ ]:
box_size = int( 400 / 2 )
n_part_per_side = int( 250 / 2 )

dx = box_size / n_part_per_side

n_part_gas = n_part_per_side**3
n_part = np.array([n_part_gas, 0, 0, 0, 0, 0])

In [ ]:
header_group.attrs["NumPart_ThisFile"] = n_part
header_group.attrs["NumPart_Total"]    = n_part
header_group.attrs["NumPart_Total_HighWord"] = np.zeros(6)

header_group.attrs["MassTable"] = np.zeros(6)

header_group.attrs["Time"] = 0.
header_group.attrs["Redshift"] = 0.
header_group.attrs["BoxSize"] = float(box_size)
header_group.attrs["NumFilesPerSnapshot"] = 1
header_group.attrs["Omega0"] = 0.0
header_group.attrs["OmegaLambda"] = 0.0
header_group.attrs["HubbleParam"] = 1.0
header_group.attrs["Flag_Sfr"] = 0
header_group.attrs["Flag_Cooling"] = 1
header_group.attrs["Flag_StellarAge"] = 0
header_group.attrs["Flag_Metals"] = 1
header_group.attrs["Flag_Feedback"] = 0
header_group.attrs["Flag_DoublePrecision"] = 0
header_group.attrs["Flag_IC_Info"] = 0




In [ ]:
float_type = "<f4"
int_type = "<u4"

particles_group.create_dataset("Coordinates",                (n_part_gas, 3), dtype=float_type)
particles_group.create_dataset("Density",                    (n_part_gas,  ), dtype=float_type)
particles_group.create_dataset("ElectronAbundance",          (n_part_gas,  ), dtype=float_type)
particles_group.create_dataset("InternalEnergy",             (n_part_gas,  ), dtype=float_type)
particles_group.create_dataset("Masses",                     (n_part_gas,  ), dtype=float_type)
particles_group.create_dataset("Metallicity",                (n_part_gas,  ), dtype=float_type)
particles_group.create_dataset("NeutralHydrogenAbundance",   (n_part_gas,  ), dtype=float_type)
particles_group.create_dataset("ParticleChildIDsNumber",     (n_part_gas,  ), dtype=int_type)
particles_group.create_dataset("ParticleIDGenerationNumber", (n_part_gas,  ), dtype=int_type)
particles_group.create_dataset("ParticleIDs",                (n_part_gas,  ), dtype=int_type)
particles_group.create_dataset("SmoothingLength",            (n_part_gas,  ), dtype=float_type)
particles_group.create_dataset("Velocities",                 (n_part_gas, 3), dtype=float_type)

In [ ]:
i = 0
for x in np.linspace(0,box_size,num=n_part_per_side+1)[1:]:
    print(x)
    for y in np.linspace(0,box_size,num=n_part_per_side+1)[1:]:
        for z in np.linspace(0,box_size,num=n_part_per_side+1)[1:]:
            particles_group["Coordinates"][i,0] = x + dx*(np.random.random()-.5)*1e-3
            particles_group["Coordinates"][i,1] = y + dx*(np.random.random()-.5)*1e-3
            particles_group["Coordinates"][i,2] = z + dx*(np.random.random()-.5)*1e-3
            i += 1

In [ ]:
# Calculate e_int

# mean molecular weight determined by initializing a snapshot with T=10**4
#  set in the params file, then looking at the InternalEnergy of snapshot_000
#  -- may not hold for other metallicities?
mu = 1.2194864
T = 1e4
gamma = 5/3

e_int = const.k_B * (T * u.Kelvin) / mu / const.m_p / (gamma-1)
e_int = e_int.to(u.pc**2 / u.Myr**2).value

In [ ]:
particles_group["Coordinates"][:] = np.array(particles_group["Coordinates"]) % box_size

density = 1.33 * m_p / cm**3
particles_group["Density"][:] = density

particles_group["ParticleIDs"][:] = np.arange(1, n_part_gas+1, dtype="<u4")

particles_group["Velocities"][:] = 0
particles_group["InternalEnergy"][:] = e_int
particles_group["Masses"][:] = dx**3 * density

## I *think* the following are correct for making this a valid snapshot file
particles_group["ElectronAbundance"][:] = 1.
particles_group["Metallicity"][:] = initial_metallicity
particles_group["NeutralHydrogenAbundance"][:] = 0.
particles_group["ParticleChildIDsNumber"][:] = 0
particles_group["ParticleIDGenerationNumber"][:] = 0
particles_group["SmoothingLength"][:] = 5.25 * (box_size/400) / (n_part_per_side/150) # approximating from existing uniform snapshots


In [ ]:
list(particles_group.keys())

In [ ]:
ic_file.close()

# Copy to outputs dir

In [ ]:
import shutil

In [ ]:
snapshot_000_filename = os.path.join(outputs_dir, "snapshot_000.hdf5")
shutil.copy2(IC_filename, snapshot_000_filename)

# Create initial energy file:

First, delete any existing energy file:

In [ ]:
energy_filename = os.path.join(outputs_dir, "energy.txt")

In [ ]:
%rm -f $energy_filename

Now create a new energy file

In [ ]:
snapshot_to_energy_file(snapshot_000_filename, energy_filename)

In [ ]:
%cat  $energy_filename

# What to do next?

 1) Add the first SNe (`add_SN_to_snapshot.ipynb`)
 
 2) Create an initial params file (`create_new_params_file.ipynb`) 
 
 3) Start simulation using the respective `*_loop_test`